<a href="https://colab.research.google.com/github/a9garza/python_fundamentals/blob/main/Automatizaci%C3%B3n_Reporte_Hechos_Relevantes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importar paquetes a utilizar

# Numpy para procesar numeros y calculos
import numpy as np

# Pandas para manejo de bases de datos
import pandas as pd

# Matplotlib para graficar
import matplotlib.pyplot as plt

# Scipy-rankdata para ordenar
from scipy.stats import rankdata

# Datetime para manejo de fechas
import datetime

import operator

# Cargando Prueba Ajustes 12345

In [ ]:
# Importar bases de datos de produccion y demoras para explicaciones
## Importar Produccion
prod_site_df = pd.read_excel ('Produccion.xlsx')

### Modificar nombre de columnas y ajustar tipo de fecha
prod_site_df.rename(columns = {'COD_DIA':'Fecha', 'COD_SUBLINEA':'codigo'}, inplace = True)
prod_site_df['Fecha'] = pd.to_datetime(prod_site_df['Fecha'], format='%Y%m%d')
prod_site_df['Fecha'] = pd.to_datetime(prod_site_df['Fecha']).dt.date
prod_site_df['DESVIO'] = prod_site_df['PRODREAL'] - prod_site_df['PRODORIGINAL']

## Importar Demoras
dem_site_df = pd.read_excel ('Demoras.xlsx')

### Modificar nombre de columnas y ajustar tipo de fecha
dem_site_df.rename(columns = {'COD_DIA':'Fecha', 'COD_SUBLINEA':'codigo'}, inplace = True)
dem_site_df['Fecha'] = pd.to_datetime(dem_site_df['Fecha'], format='%Y%m%d')
dem_site_df['Fecha'] = pd.to_datetime(dem_site_df['Fecha']).dt.date
dem_site_df = dem_site_df.replace('Reparación general (rex)','REX')
dem_site_df = dem_site_df.replace('Reparación/mantenimiento programado','RP')

## Importar tablas de Apoyo
### Estructura de Lineas
est_lineas_df = pd.read_excel ('Estructura Lineas.xlsx')
### Estructura de Demoras
est_dem_df = pd.read_excel ('Estructura Demoras.xlsx')
est_dem_df= est_dem_df.replace('Reparación general (rex)','REX')
est_dem_df= est_dem_df.replace('Reparación/mantenimiento programado','RP')

### listado de líneas
lineas_df = pd.read_excel ('Lineas DICO.xlsx')

In [ ]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
lineas_list = est_lineas_df[['COD_PROCESO','DES_PROCESO','COD_SUBLINEA','DES_SUBLINEA','NOMBRELINEACORTO','REGION','PLANTA','UNIDADNEGOCIO']].values.tolist()
lineas_df_list = lineas_df[['Nombre correo','Órden','codigo']].values.tolist()
indicadores_list =['Ut. Disp.', 'Ut. Neta','Ef. Neta','Mix']
tipo_unique = dem_site_df['COD_DEMORA'].unique()
rubro_unique = dem_site_df['COD_DIVISION'].unique()
subrubro_unique = dem_site_df['COD_SUBDIVISION'].unique()

#Creacion de diccionarios de demoras por nivel
tipo_dict = {tipo:list(dem_site_df[dem_site_df.COD_DEMORA == tipo]['DES_DEMORA'].unique()) for tipo in tipo_unique}
rubro_dict = {rubro:list(dem_site_df[dem_site_df.COD_DIVISION == rubro]['DES_DIVISION'].unique()) for rubro in rubro_unique}
subrubro_dict = {subrubro:list(dem_site_df[dem_site_df.COD_SUBDIVISION == subrubro]['DES_SUBDIVISION'].unique()) for subrubro in subrubro_unique}
diccionarios = [tipo_dict,rubro_dict,subrubro_dict]


#Declaracion de variables inicial
hrs_std = []
hrs_real = []
hrs_desvio = []
desvios_linea = []
cod_name = []

In [ ]:
# FUNCIONES SECUNDARIAS

# Funcion para buscar codigo de linea - Buscar Agregar dentro de cada línea, al generar la línea por clase
def cod_linea(linea,lineas_list):
    try:
        for i in range(len(lineas_list)):
                if linea in lineas_list[i]:
                 return int(lineas_list[i][2])
    except:
        return "Linea {} no existe en diccionario".format(linea)



#Funcion para buscar codigo de proceso
def cod_proceso(proceso,lineas_list):
    for i in range(len(lineas_list)):
            if proceso in lineas_list[i]:
             return lineas_list[i][0]
    return "Linea no existe en diccionario"



#Funcion para buscar par codigo,nombre de demoras
def cod_demora(codigo,diccionario):
    for key,value in diccionario.items():
        if codigo == key:
            return key,value
    return 'Error en cod_demora'

In [ ]:
# Función para establecer que líneas se deben de explicar en el rango de fechas establecido
def lineas_DICO(fecha_inicio,fecha_fin,prod_site_df):
    startdate = pd.to_datetime(fecha_inicio).date()
    enddate = pd.to_datetime(fecha_fin).date()
    filtered_prod_site_df = prod_site_df.loc[(prod_site_df['Fecha'] >= startdate)& (prod_site_df['Fecha'] <= enddate)]
    df_listado = pd.DataFrame(filtered_prod_site_df.groupby(['codigo','DES_SUBLINEA'])["DESVIO"].sum())
    df_listado = df_listado[df_listado['DESVIO'] <= -250]
    df_listado = pd.merge(df_listado, lineas_df, how="left", on=["codigo", "codigo"])
    df_listado = df_listado[df_listado['Órden'].notnull()]
    df_listado = df_listado.sort_values(by=['Órden']).reset_index(drop=True)
    codigos_DICO = df_listado[['codigo','Nombre correo']].values.tolist()
    return codigos_DICO

In [ ]:
#Funcion para explicaciones de desvios por Utilizacion Disponible
def paradas(desvios_linea,pdd_disp_std,desvio_udisp,desvios_paradas_hrs):
    desvios_paradas_neg_texto = ''
    desvios_paradas_pos_texto = ''
    desvios_paradas_final_texto = ''
    desvios_paradas = [(i,j,k,l,l/60*pdd_disp_std) for i,j,k,l in desvios_paradas_hrs]
    desvios_paradas_pos = [(i,j,k,l,m) for i,j,k,l,m in desvios_paradas if m>50]
    desvios_paradas_neg = [(i,j,k,l,m) for i,j,k,l,m in desvios_paradas if m<-50]

    if desvio_udisp<0:
        desvios_tot_paradas_pos = sorted(desvios_paradas_pos, key = operator.itemgetter(3),reverse=True)
        desvios_tot_paradas_neg = sorted(desvios_paradas_neg, key = operator.itemgetter(3),reverse=False)
        for i in range(len(desvios_tot_paradas_neg)):
            desvios_paradas_neg_texto += ' por {} ({}hrs./{}kt),'.format(desvios_tot_paradas_neg[i][0][1][0],round(desvios_tot_paradas_neg[i][3]/60,1),round(desvios_tot_paradas_neg[i][4]/1000,1))
        for i in range(len(desvios_tot_paradas_pos)):
            desvios_paradas_pos_texto += ' por {} (+{}hrs./+{}kt),'.format(desvios_tot_paradas_pos[i][0][1][0],round(desvios_tot_paradas_pos[i][3]/60,1),round(desvios_tot_paradas_pos[i][4]/1000,1))
        if len(desvios_paradas_pos) >0:
            desvios_paradas_final_texto = '{}. Gana{}'.format(desvios_paradas_neg_texto[:-1],desvios_paradas_pos_texto[:-1])
        else:
            desvios_paradas_final_texto = '{}'.format(desvios_paradas_neg_texto[:-1])
    elif desvio_udisp>0:
        desvios_tot_paradas_pos = sorted(desvios_paradas_pos, key = operator.itemgetter(3),reverse=True)
        desvios_tot_paradas_neg = sorted(desvios_paradas_neg, key = operator.itemgetter(3),reverse=False)
        for i in range(len(desvios_tot_paradas_neg)):
            desvios_paradas_neg_texto += ' por {} ({}hrs./{}kt),'.format(desvios_tot_paradas_neg[i][0][1][0],round(desvios_tot_paradas_neg[i][3]/60,1),round(desvios_tot_paradas_neg[i][4]/1000,1))
        for i in range(len(desvios_tot_paradas_pos)):
            desvios_paradas_pos_texto += ' por {} (+{}hrs./+{}kt),'.format(desvios_tot_paradas_pos[i][0][1][0],round(desvios_tot_paradas_pos[i][3]/60,1),round(desvios_tot_paradas_pos[i][4]/1000,1))
        if len(desvios_paradas_neg)>0:
            desvios_paradas_final_texto = '{}. Pierde{}'.format(desvios_paradas_pos_texto[:-1],desvios_paradas_neg_texto[:-1])
        else:
            desvios_paradas_final_texto = '{}'.format(desvios_paradas_pos_texto[:-1],desvios_paradas_neg_texto[:-1])
    return desvios_paradas_final_texto


#Funcion para explicaciones de desvios por Utilizacion Neta
def interrupciones(desvios_linea,pdd_net_std,desvio_unet,desvios_int_hrs):
    desvios_int_neg_texto = ''
    desvios_int_pos_texto = ''
    desvios_int_final_texto = ''
    desvios_int = [(i,j,k,l,l/60*pdd_net_std) for i,j,k,l in desvios_int_hrs]
    desvios_int_pos = [(i,j,k,l,m) for i,j,k,l,m in desvios_int if m>50]
    desvios_int_neg = [(i,j,k,l,m) for i,j,k,l,m in desvios_int if m<-50]



    if desvio_unet<0:
        desvios_tot_int_pos = sorted(desvios_int_pos, key = operator.itemgetter(3),reverse=True)
        desvios_tot_int_neg = sorted(desvios_int_neg, key = operator.itemgetter(3),reverse=False)
        for i in range(len(desvios_tot_int_neg)):
            desvios_int_neg_texto += ' por {} ({}hrs./{}kt),'.format(desvios_tot_int_neg[i][0][1][0],round(desvios_tot_int_neg[i][3]/60,1),round(desvios_tot_int_neg[i][4]/1000,1))
        for i in range(len(desvios_tot_int_pos)):
            desvios_int_pos_texto += ' por {} (+{}hrs./+{}kt),'.format(desvios_tot_int_pos[i][0][1][0],round(desvios_tot_int_pos[i][3]/60,1),round(desvios_tot_int_pos[i][4]/1000,1))
        if len(desvios_int_pos) >0:
            desvios_int_final_texto = '{}. Gana{}'.format(desvios_int_neg_texto[:-1],desvios_int_pos_texto[:-1])
        else:
            desvios_int_final_texto = '{}'.format(desvios_int_neg_texto[:-1])
    elif desvio_unet>0:
        desvios_tot_int_pos = sorted(desvios_int_pos, key = operator.itemgetter(3),reverse=True)
        desvios_tot_int_neg = sorted(desvios_int_neg, key = operator.itemgetter(3),reverse=False)
        for i in range(len(desvios_tot_int_neg)):
            desvios_int_neg_texto += ' por {} ({}hrs./{}kt),'.format(desvios_tot_int_neg[i][0][1][0],round(desvios_tot_int_neg[i][3]/60,1),round(desvios_tot_int_neg[i][4]/1000,1))
        for i in range(len(desvios_tot_int_pos)):
            desvios_int_pos_texto += ' por {} (+{}hrs./+{}kt),'.format(desvios_tot_int_pos[i][0][1][0],round(desvios_tot_int_pos[i][3]/60,1),round(desvios_tot_int_pos[i][4]/1000,1))
        if len(desvios_int_neg)>0:
            desvios_int_final_texto = '{}. Pierde{}'.format(desvios_int_pos_texto[:-1],desvios_int_neg_texto[:-1])
        else:
            desvios_int_final_texto = '{}'.format(desvios_int_pos_texto[:-1],desvios_int_neg_texto[:-1])
    return desvios_int_final_texto

In [ ]:
# Funcion para calculo de desvios de produccion y estructuras de tiempo de cada linea
def explicaciones_desvios(startdate,enddate,codigo_linea,linea,prod_site_df,dem_site_df):
    #Filtro de bases de datos para tomar 1 sola linea; Bases de Produccion y Demoras
    demoras_df = dem_site_df.loc[(dem_site_df['Fecha'] >= startdate)
                     & (dem_site_df['Fecha'] <= enddate)& (dem_site_df['codigo'] == codigo_linea)]
    produccion_df = prod_site_df.loc[(prod_site_df['Fecha'] >= startdate)
                     & (prod_site_df['Fecha'] <= enddate)& (prod_site_df['codigo'] == codigo_linea)]
    #Declaracion de variables inicial
    hrs_std = []
    hrs_real = []
    hrs_desvio = []
    desvios_linea = []
    cod_name = []
    # Crear lista de desvios en formato -> ((Codigo,Nombre), STD, Real, Desvio hrs, desvio tns)
    for dicts in diccionarios:
        for key,value in dicts.items():
            hrs_std += [demoras_df.loc[(demoras_df['COD_DEMORA']==key)|(demoras_df['COD_DIVISION']==key)|(demoras_df['COD_SUBDIVISION']==key)]['MINUTOSSTDDIA'].sum()]
            hrs_real += [demoras_df.loc[(demoras_df['COD_DEMORA']==key)|(demoras_df['COD_DIVISION']==key)|(demoras_df['COD_SUBDIVISION']==key)]['MINUTOSREALDIA'].sum()]
            hrs_desvio = [std-real for (std,real) in zip(hrs_std,hrs_real)]
            cod_name.append(cod_demora(key,dicts))
            desvios_linea = list(zip(cod_name,hrs_std,hrs_real,hrs_desvio))



#Obtener info de demoras para calculo de estructuras de tiempos - info en minutos
    paradas_real = [item[2] for item in desvios_linea if item[0][0] == 2000][0]
    paradas_std = [item[1] for item in desvios_linea if item[0][0] == 2000][0]
    interrupciones_real = [item[2] for item in desvios_linea if item[0][0] == 1000][0]
    interrupciones_std = [item[1] for item in desvios_linea if item[0][0] == 1000][0]
    estructurales_real = [item[2] for item in desvios_linea if item[0][0] == 2400][0]
    estructurales_std = [item[1] for item in desvios_linea if item[0][0] == 2400][0]
    rex_real = [item[2] for item in desvios_linea if item[0][0] == 2220][0]
    rex_std = [item[1] for item in desvios_linea if item[0][0] == 2220][0]



#Tiempo calendario en base al numero de dias inicial, final
    tcal = ((enddate-startdate).days+1)*24



#Calculo de estructura de tiempos real - informacion en horas
    tdisp_real = tcal-(paradas_real/60)
    tpos_real = tcal-(rex_real/60)-(estructurales_real/60)
    tnet_real = tdisp_real - (interrupciones_real/60)
    utot_real = tnet_real/tcal
    udisp_real = tdisp_real/tcal
    upos_real = tnet_real/tpos_real
    unet_real = tnet_real/tdisp_real



#Calculo de estructura de tiempos std - informacion en horas
    tdisp_std = tcal-(paradas_std/60)
    tpos_std = tcal-(rex_std/60)-(estructurales_std/60)
    tnet_std = tdisp_std - (interrupciones_std/60)
    tnet_flex = (produccion_df['TIEMPONETOSTDFLEX'].sum())/60
    utot_std = tnet_std/tcal
    udisp_std = tdisp_std/tcal
    upos_std = tnet_std/tpos_std
    unet_std = tnet_std/tdisp_std
    unet_flex = tnet_flex/tdisp_std
    tdisp_flex = tnet_flex/unet_flex

# Calculo de info de produccion
    prod_real = produccion_df['PRODREAL'].sum()
    prod_orig = produccion_df['PRODORIGINAL'].sum()
    coef_prog = produccion_df['COEFICIENTEPROG'].mean()
    desvio_tns = prod_real-prod_orig
    pdd_disp_real = prod_real/tdisp_real
    pdd_net_real = prod_real/tnet_real
    pdd_disp_obj = prod_orig/tdisp_std
    pdd_disp_std = prod_real/tdisp_std # multiplicar por coeficiente de programacion
    pdd_net_obj = prod_orig/tnet_std
    pdd_net_std = prod_real/tnet_flex
    pdd_disp_flex = pdd_net_std*unet_std
    #pdd_disp_flex = prod_real/tnet_flex


# Calculo de desvio de indicadores
    desvio_mix = tdisp_std*(pdd_disp_flex-pdd_disp_obj) # multiplicar por coeficiente
    desvio_udisp = (tdisp_real-tdisp_std)*pdd_disp_flex # multiplicar por coeficiente
    #desvio_coef = tdisp_real*pdd_disp_flex*(1-coef_prog)
    desvio_unet = tdisp_real*(unet_real-unet_std)*pdd_net_std
    desvio_pddnet = tdisp_real*unet_real*(pdd_net_real-pdd_net_std)
    desvio_tot = desvio_mix+desvio_udisp+desvio_unet+desvio_pddnet
    desvios = [desvio_udisp,desvio_unet,desvio_pddnet,desvio_mix]
    desvios_int_hrs = [item for item in desvios_linea if (item[0][0] < 2000) & (item[0][0]%100 != 0)]
    desvios_paradas_hrs = [item for item in desvios_linea if (item[0][0] > 2000) & (item[0][0]%100 != 0)]
    explicaciones = explicaciones_linea(prod_real,desvio_tns,codigo_linea,linea,indicadores_list,desvios,desvio_udisp,desvios_paradas_hrs,pdd_disp_std,desvio_unet,desvios_int_hrs,pdd_net_std)
    #prueba_texto = 'Tiempo Calendario: {}\nTiempo Disponible Real: {}\nTiempo Disponible STD: {}\nTiempo Neto Real: {}\nTiempo Neto STD: {}\nPdd. Disp.: {}\nPdd. Neta: {}\nCoef.: {}'.format(tcal,tdisp_real,tdisp_std,tnet_real,tnet_flex,pdd_disp_flex,pdd_net_std,coef_prog)
    #print(prueba_texto)
    return explicaciones

In [ ]:
def explicaciones_linea(prod_real,desvio_tns,codigo_linea,linea,indicadores_list,desvios,desvio_udisp,desvios_paradas_hrs,pdd_disp_std,desvio_unet,desvios_int_hrs,pdd_net_std):
    explicaciones_paradas = paradas(desvios_linea,pdd_disp_std,desvio_udisp,desvios_paradas_hrs)
    explicaciones_int = interrupciones(desvios_linea,pdd_net_std,desvio_unet,desvios_int_hrs)
    zipped = list(zip(indicadores_list,desvios))
    desvios_pos = [(i,j) for (i,j) in zipped if j>=0]
    desvios_neg = [(i,j) for (i,j) in zipped if j<0]
    desvios_final = ''
    formato_END=color.END
    apertura = ""
    anexo =""
    intro = ": "
    formato_linea = color.BOLD
    if desvio_tns == 0:
        formato_color = ''
        if prod_real ==0 & prod_std == 0:
            print("La linea no tuvo producción")
        else:
            print("La línea no tiene desvíos")
    elif desvio_tns <0:
        signo = '-'
        formato_color = color.RED
        desvios_tot = sorted(zipped, key = operator.itemgetter(1),reverse=False)
        for i in range(len(desvios_tot)):
            if desvios_tot[i][1]>=0:
                signo_ind = '+'
                color_ind=''
                if desvios_tot[i][0]=='Ut. Neta':
                    desvios_final += str(desvios_tot[i][0])+color_ind+' ('+str(signo_ind)+str(round(desvios_tot[i][1]/1000,1))+' kt)'+color.END+'; '
                elif desvios_tot[i][0]=='Ut. Disp.':
                    desvios_final += str(desvios_tot[i][0])+color_ind+' ('+str(signo_ind)+str(round(desvios_tot[i][1]/1000,1))+' kt)'+color.END+'{}; '.format(explicaciones_paradas)
                else:
                    desvios_final += str(desvios_tot[i][0])+color_ind+' ('+str(signo_ind)+str(round(desvios_tot[i][1]/1000,1))+' kt)'+color.END+'; '



            else:
                signo_ind=''
                color_ind=color.RED
                if desvios_tot[i][0]=='Ut. Neta':
                    desvios_final += str(desvios_tot[i][0])+color_ind+' ('+str(signo_ind)+str(round(desvios_tot[i][1]/1000,1))+' kt)'+color.END+'{}; '.format(explicaciones_int)
                elif desvios_tot[i][0]=='Ut. Disp.':
                    desvios_final += str(desvios_tot[i][0])+color_ind+' ('+str(signo_ind)+str(round(desvios_tot[i][1]/1000,1))+' kt)'+color.END+'{}; '.format(explicaciones_paradas)
                else:
                    desvios_final += str(desvios_tot[i][0])+color_ind+' ('+str(signo_ind)+str(round(desvios_tot[i][1]/1000,1))+' kt)'+color.END+' {}; '



    elif desvio_tns>0:
        signo = '+'
        formato_color = ''
        desvios_tot = sorted(zipped, key = operator.itemgetter(1),reverse=True)
        for i in range(len(desvios_tot)):
            if desvios_tot[i][1]>=0:
                signo_ind = '+'
                color_ind=''
            else:
                signo_ind=''
                color_ind=color.RED
            if desvios_tot[i][0]=='Ut. Neta':
                desvios_final += str(desvios_tot[i][0])+color_ind+' ('+str(signo_ind)+str(round(desvios_tot[i][1]/1000,1))+' kt)'+color.END+'{}; '.format(explicaciones_int)
            elif desvios_tot[i][0]=='Ut. Disp.':
                desvios_final += str(desvios_tot[i][0])+color_ind+' ('+str(signo_ind)+str(round(desvios_tot[i][1]/1000,1))+' kt)'+color.END+'{}; '.format(explicaciones_paradas)
            else:
                desvios_final += str(desvios_tot[i][0])+color_ind+' ('+str(signo_ind)+str(round(desvios_tot[i][1]/1000,1))+' kt)'+color.END+' {}; '
    texto = '{}{}{} {}({}{} kt/{}{}%){}{}{}{}.\n'.format(formato_linea,linea,formato_color,apertura,signo,abs(round(desvio_tns/1000,1)),signo,round(abs(desvio_tns/prod_real)*100,1),formato_END,intro,anexo,desvios_final[:-2])
    return print(texto)

In [ ]:
# Función Principal
def explicaciones_DICO(fecha_inicio,fecha_fin,prod_site_df,dem_site_df):
    formato_linea = color.BOLD
    formato_END=color.END
    explicaciones_final=[]
    startdate = pd.to_datetime(fecha_inicio).date()
    enddate = pd.to_datetime(fecha_fin).date()
    codigos_DICO = lineas_DICO(fecha_inicio,fecha_fin,prod_site_df)
    detalle = 'Número de Líneas: {} líneas\n'.format(len(codigos_DICO))
    #result = startdate.month_name(locale = 'English')
    #Nos da la lista de lineas -> Sale un listado de codigos y nombres
    #Entra un listado de codigos e itera la sig explicaciones
    print('{}{}Hechos Relevantes ({} al {} de {} de {}){}\n'.format(detalle,formato_linea,startdate.day,enddate.day,enddate.strftime("%B"),enddate.year,formato_END))
    for i in range(len(codigos_DICO)):
        codigo_linea = codigos_DICO[i][0]
        linea = codigos_DICO[i][1]
        try:
            explicaciones = explicaciones_desvios(startdate,enddate,codigo_linea,linea,prod_site_df,dem_site_df)
        except:
            print('La linea {} tiene errores'.format(linea))
        #explicaciones_final += explicaciones_desvios(startdate,enddate,codigo_linea,linea,prod_site_df,dem_site_df)
        #Calcula la explicacion pora cada linea
    return explicaciones
#explicaciones_DICO('2022-08-18','2022-08-23',prod_site_df,dem_site_df)

In [ ]:
explicaciones_DICO('2023-06-06','2023-06-12',prod_site_df,dem_site_df)

Número de Líneas: 6 líneas
Hechos Relevantes (6 al 12 de June de 2023)

Galva 1 JUV (-0.3 kt/-9.0%): Ut. Disp. (-0.1 kt) por Cambios operativos (-4.2hrs./-0.1kt); Mix (-0.1 kt) {}; Ef. Neta (-0.1 kt) {}; Ut. Neta (-0.1 kt).

Galva 3 UNI (-0.8 kt/-21.0%): Ef. Neta (-0.6 kt) {}; Ut. Disp. (-0.6 kt) por Falta de materia prima (-20.4hrs./-0.5kt), por Cambio de producto o medida (-15.2hrs./-0.4kt). Gana por REX (+16.0hrs./+0.4kt), por Cambios operativos (+3.5hrs./+0.1kt); Ut. Neta (+0.0 kt); Mix (+0.4 kt).

Galva Mva (-0.4 kt/-7.4%): Ef. Neta (-0.2 kt) {}; Ut. Neta (-0.2 kt) por Eléctricas (-4.7hrs./-0.2kt); Mix (-0.1 kt) {}; Ut. Disp. (+0.0 kt).

Galva 1 PES (-0.7 kt/-9.1%): Mix (-0.9 kt) {}; Ef. Neta (-0.2 kt) {}; Ut. Neta (+0.1 kt); Ut. Disp. (+0.3 kt) por Cambios operativos (+5.0hrs./+0.2kt), por Pruebas y ensayos (+1.5hrs./+0.1kt).

Pintado 3 JUV (-0.6 kt/-19.1%): Ut. Disp. (-0.4 kt) por Cambios operativos (-12.9hrs./-0.3kt), por Cambio de producto o medida (-3.2hrs./-0.1kt); Mix (-0.1

In [ ]:
# Base de Datos de Demoras -> SGL
df_demoras = pd.read_excel ('demoras.xlsx')

# Convertir las fechas al formato datetime
date_columns = ['Fecha Inicio', 'Fecha Fin']
df_demoras[date_columns] = df_demoras[date_columns].apply(pd.to_datetime, format='%d/%m/%y %H:%M')

# Ordenar las bases de datos por Línea y fecha
sorted_demoras = df_demoras.sort_values(by=['Linea', 'Fecha Inicio'])

In [ ]:
print(codigos_DICO)

NameError: ignored